In [5]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
import requests
import dill # to store 
from bs4 import BeautifulSoup # for HTML scrapping 
from datetime import datetime
import pandas as pd

import numpy as np # library to handle data in a vectorized manner
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library
from scipy.stats import zscore
from math import radians, cos, sin, asin, sqrt,pi, sin, atan2,acos
from sklearn import preprocessing
import json # library to handle JSON files
from shapely.geometry import shape, Point
from collections import defaultdict
import pickle


#### loading any previously stored dataframe and check the locations of the stored listings in order to add new ones

In [ ]:
with open("datframe1.txt", "rb") as fp:   # Unpickling
    dff= pickle.load(fp)

locs=[l.strip() for l in dff['location']]

#### scrapping the website to find the available listings

In [ ]:
#using BeautifulDoup
url= 'https://www.spacelist.ca/listings/on/toronto/retail/for-lease'
page =  requests.get(url) # Use requests.get to download the page.
soup = BeautifulSoup(page.text,'html.parser')

listings=[]
for element in soup.find_all('div', class_='meta-card'):
    listings.append(element)


#### repeate for all pages

In [ ]:
for i in range(2,11):
    url= f'https://www.spacelist.ca/listings/on/toronto/retail/for-lease/page/{i}'
    page =  requests.get(url) # Use requests.get to download the page.
    soup = BeautifulSoup(page.text,'html.parser')

    for element in soup.find_all('div', class_='meta-card'):
        listings.append(element)
    

In [ ]:
len(listings)

### Creating the datframe: by extracting the location, lease rate and the space of each listing

In [ ]:
spaces=[]
space_idx=[]
rents=[]
rent_idx=[]
locations=[]

for listing in listings:
    rent= listing.find('div',class_='cell shrink display-price')
    space=listing.find('div',class_='cell auto about')
    location=listing.find('div',class_='cell auto small-font dark-font flat-line-height right-cell-padding')
    if type(rent) != type(None)  and type(space) != type(None) and type(location) != type(None) :
    
        text2= space.text
        text=rent.text
        text3=location.text
    
        if ',' in text:text=text.replace(',','')
        if 'Sale' in text:text=text.replace('Sale','')
        if '$' in text:text=text.replace('$','')
        if ',' in text:text=text.replace(',','')
        if '-' in text:text=text.split('-')[-1].strip()
        #take the monthly rate only and handling the different formats of text
        
        if 'yr' in text:text= text.split('yr')[-1].strip()
        if '/mo' in text: text= text.split('/mo')[0].strip()
        if ',' in text2:text2=text2.replace(',','')
        if '-' in text2:
            space_ft2=text2.split('-')[0].strip()   
        elif ('ft' in text2):
            space_ft2=text2.split('ft')[0].strip()
        elif('Desk'in text2):
            space_ft2='25'
            
        if '-'in text3:place=text3.split('-')[-1]
        else: place=text3
        if place.strip() in locs:continue
        rents.append(text)
        spaces.append(space_ft2)
        locations.append(place)
    
    

##### removing data with no price info and finding the price per sqft

In [ ]:
df = pd.DataFrame(list(zip(rents,spaces,locations)), columns =['rent','spaces','location']) 
df=df.drop(df[df['rent']=='ContactContactLease'].index)
df=df.drop(df[df['rent']=='ContactContactLease '].index)
df=df.drop( df[df['rent']=='ContactContactLease (NNN)'].index ) 
df=df.drop( df[df['rent']=='ContactContactLease (Net)'].index ) 
df=df.drop(df[df['rent']=='ContactContactSublease'].index)
df=df.drop(df[df['rent']=='ContactContactSublease '].index)


In [ ]:
df=df.reset_index(drop=True)
df['price_per_sqft']=df['rent'].astype(float)/df['spaces'].astype(float)
#df.head()

In [ ]:
len(df)

##### adding long and lat info for all the listings using geolocator

In [ ]:
df['latitude'] = ""
df['longitude']=""

for i in range(0,len(df)):
    address = df['location'][i]+ ', Toronto, canada'
    geolocator = Nominatim(user_agent="ny_explorer",timeout=3)
    location = geolocator.geocode(address)
    if location:
        df['latitude'][i]= location.latitude
        df['longitude'][i] = location.longitude
  

#### Adding subway stations information from City of Toronto API

In [ ]:
subway=pd.read_csv('TorontoSubwayStations_Ridership.csv')
subway.rename(columns={"y": "latitude", "x": "longitude",
                       "Station Name": "station_name",
                       "Total Daily Riders":"daily_riders"}, errors="raise", inplace=True)
subway=subway.reset_index(drop=True)

In [ ]:
x = subway[['daily_riders']].values.astype(float)
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(1, 10))
# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)
subway['scaled_riders']=x_scaled

###### distacne calculator function 

In [ ]:
def distance(lat1, lon1, lat2, lon2):
    R = 6371
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    phi=(lat1+lat2)/2

    a = (dLat**2)+(cos(phi)*dLon)**2
    c = (sqrt(a))

    return (R * c)

##### finding the distance between each listing to the anearest subway station and adding it to the dataframe:

In [ ]:
df['nearest_subway']=""
df['dist_to_subway_km'] = ""

for i in range(len(df)):
    distances=[]
    for lat,long in zip(subway['latitude'], subway['longitude']):
        distances.append(distance(df['latitude'][i],df['longitude'][i],lat,long))
        
    x=min(distances)
    for j in range(len(distances)):
        if distances[j]==x:
            nearest=subway.iloc[j]['station_name']
            daily_riders=subway.iloc[j]['daily_riders']

    df['dist_to_subway_km'][i]=min(distances)
    df['nearest_subway'][i]=nearest
    df['subway_daily_riders']=daily_riders
    

##### finding the distance to the nearest bus stop

In [ ]:
stops=pd.read_csv('stops.csv')
df['dist_to_bus_km'] = ""

for i in range(len(df)):
    distances=[]
    for lat,long in zip(stops['stop_lat'], stops['stop_lon']):
        distances.append(distance(df['latitude'][i],df['longitude'][i],lat,long))

    df['dist_to_bus_km'][i]=min(distances)


### Now we find the neihbourhood of each listing to add the neihbourhood information to the data frame using shapely.geometry library

In [ ]:
#loading a geojson file that has the bounderies of each neihbourhood
with open('Neighbourhoods.geojson.json') as f:
    js = json.load(f)

In [ ]:
codes=[]
for ind in df.index: 
    point= Point(df['longitude'][ind],df['latitude'][ind])
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point):
                codes.append(feature['properties']['AREA_SHORT_CODE'])

df['AREA_SHORT_CODE']=codes


#### more info about each neighborhood will be added

In [ ]:
file = 'population10_89.csv'
population = pd.read_csv(file)
population.drop('Combined Indicators',1,inplace=True)
file = 'economics _toronto.csv'
economics = pd.read_csv(file)
economics.drop('Combined Indicators',1,inplace=True)

In [ ]:
file = 'Neighbourhood_Crime_Rates_Boundary_File_.csv'
crime_data = pd.read_csv(file)
columns=['Hood_ID','Neighbourhood','Assault_AVG','Assault_CHG','AutoTheft_AVG','AutoTheft_CHG','BreakandEnter_AVG','BreakandEnter_CHG','Robbery_AVG','Robbery_CHG','heftOver_AVG','TheftOver_CHG','Homicide_AVG','Homicide_CHG','Population','Shape__Area','Shape__Length']
crime=crime_data.loc[:,columns]
data_df=population.merge(economics,  how='left', left_on=['Neighbourhood Id','Neighbourhood'],
                         right_on=['Neighbourhood Id','Neighbourhood'])
data_df=data_df.merge(crime,how='left', left_on=['Neighbourhood Id','Neighbourhood'],
                      right_on=['Hood_ID','Neighbourhood'])
columns=['Assault_CHG','AutoTheft_CHG','Hood_ID','BreakandEnter_CHG','Robbery_CHG','Homicide_AVG','Population','heftOver_AVG','TheftOver_CHG','Homicide_CHG']
data_df.drop(columns,axis=1,inplace=True)


#### grouping the different age populations into 4 age groups that could be relevent to the different businesses

In [ ]:
total_pop=[]
Pop_10_24=[]
Pop_25_39=[]
Pop_40_59=[]
Pop_60_89=[]
Pop_Males=[]
Pop_Females=[]
avg_crime=[]
Low_Income_Families=[]
Part_Time_Employment=[]
shape_area=[]
for i in range(len(df)):
    Nid=int(df.iloc[i]['AREA_SHORT_CODE'])
    total_pop.append(data_df[data_df['Neighbourhood Id']==Nid]['Total Population'].values[0])
    Pop_10_24.append(data_df[data_df['Neighbourhood Id']==Nid]['Pop 10 - 14 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 15 - 19 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 20 - 24 years'].values[0])
    Pop_25_39.append(data_df[data_df['Neighbourhood Id']==Nid]['Pop 25 - 29 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 30 - 34 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 35 - 39 years'].values[0])
    Pop_40_59.append(data_df[data_df['Neighbourhood Id']==Nid]['Pop 40 - 44 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 45 - 49 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 50 - 54 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 55 - 59 years'].values[0])
    Pop_60_89.append(data_df[data_df['Neighbourhood Id']==Nid]['Pop 60 - 64 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 65 - 69 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 70 - 74 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 75 - 79 years'].values[0] +data_df[data_df['Neighbourhood Id']==Nid]['Pop 80 - 84 years'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Pop 85 - 89 years'].values[0])
    
    Pop_Males.append(data_df[data_df['Neighbourhood Id']==Nid]['Pop - Males'].values[0])
    Pop_Females.append(data_df[data_df['Neighbourhood Id']==Nid]['Pop - Females'].values[0])
    avg_crime.append(data_df[data_df['Neighbourhood Id']==Nid]['AutoTheft_AVG'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['BreakandEnter_AVG'].values[0]+data_df[data_df['Neighbourhood Id']==Nid]['Robbery_AVG'].values[0])
    
    Low_Income_Families.append(data_df[data_df['Neighbourhood Id']==Nid]['Low Income Families'].values[0])
    Part_Time_Employment.append(data_df[data_df['Neighbourhood Id']==Nid]['Part-Time Employment'].values[0])
    shape_area.append(data_df[data_df['Neighbourhood Id']==Nid]['Shape__Area'].values[0])
    
df['Total_Population']=total_pop
df['Pop_10-24']=Pop_10_24
df['Pop_25-39']=Pop_25_39
df['Pop_40-59']=Pop_40_59
df['Pop_60-89']=Pop_60_89
df['Pop-Males']=Pop_Males
df['Pop-Females']=Pop_Females
df['avg_crime']=avg_crime
df['Low_Income_Families']=Low_Income_Families
df['Part_Time_Employment']=Part_Time_Employment
df['Neighbor_area']=shape_area
    

### Now exploring sourounding venues using Foursqure API

###### finding nearby cafes:

In [ ]:
##Foursquare API
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET = # your Foursquare Secret
VERSION = '20200202'
LIMIT = 10000
radius = 500

In [ ]:
search_query = 'cafe'
radius = 500
cafe=[]
for i in range(len(df)):
    latitude=df.iloc[i]['latitude']
    longitude=df.iloc[i]['longitude']
    # forsquare data
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    cafe.append(len(venues))
df['cafes_within_500m']=cafe


###### Finding nearby colleges:

In [ ]:
search_query = 'college'
radius = 1000
college=[]
for i in range(len(df)):
    latitude=df.iloc[i]['latitude']
    longitude=df.iloc[i]['longitude']
    # forsquare data
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    if venues :college.append(len(venues))
    else: college.append(0)
    
df['colleges_within_1000m']=college
    

###### Finding nearby restaurants:

In [ ]:
search_query = 'food'
radius = 500
food=[]
for i in range(len(df)):
    latitude=df.iloc[i]['latitude']
    longitude=df.iloc[i]['longitude']
    # forsquare data
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    if venues :food.append(len(venues))
    else: food.append(0)
    
df['food_within_500m']=food
    

##### Finding nearby Medical centers:

In [ ]:
search_query = 'Medical Center'
radius = 1000
hospital=[]
for i in range(len(df)):
    latitude=df.iloc[i]['latitude']
    longitude=df.iloc[i]['longitude']
    # forsquare data
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    if venues :hospital.append(len(venues))
    else: hospital.append(0)
    
df['hospitals_within_1000m']=hospital
    

##### Finding nearby Shopping centers

In [ ]:
search_query = 'shopping'
radius = 1500
shopping=[]
for i in range(len(df)):
    latitude=df.iloc[i]['latitude']
    longitude=df.iloc[i]['longitude']
    # forsquare data
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    if venues :shopping.append(len(venues))
    else: shopping.append(0)
    
df['shopping_within_1500m']=shopping
    

##### Finding nearby fun centers:

In [ ]:
search_query = 'fun'
radius = 500
fun=[]
for i in range(len(df)):
    latitude=df.iloc[i]['latitude']
    longitude=df.iloc[i]['longitude']
    # forsquare data
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    venues = results['response']['venues']
    if venues :fun.append(len(venues))
    else: fun.append(0)
    
df['fun_within_500m']=fun
    

### Exploring surrounding venues to find the average ratings, no. of check-ins, and  tips

In [1]:
def getNearbyids(latitude, longitude, radius=200):
    """
    function that returns a list of venues and their ids 
    within a specific radius from a given lat and long
    """
    venues_list=[]
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
     
    results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
    return [v['venue']['id'] for v in results]
    
    

In [ ]:
def get_venue_info(ids,venues):
    """
    this function adds venue information(tips, rating) to a list of venues
    """
    rating_sum=0
    tips_sum=0
    for vid in ids:
        if vid not in venues.keys():
            url='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(vid, CLIENT_ID, CLIENT_SECRET, VERSION)
            results = requests.get(url).json()
            if 'rating' in results['response']['venue'].keys():
                rating=results['response']['venue']['rating']
            else: rating=0
            tips=results['response']['venue']['stats']['tipCount']
            rating_sum+=rating
            tips_sum+=tips
            venues[vid]=[rating,tips]
    return venues


In [2]:
def listing_nearby(ids,D):
    """
    return the average of ratings and total tips
    """
    rating_sum=0
    tips_sum=0
    for vid in ids:
        rating_sum+=D[vid][0]
        tips_sum+=D[vid][1]
    
    if len(ids)>0:return(rating_sum/len(ids),tips_sum)
    else: return(0,0)
        
        

In [15]:
##Foursquare API
CLIENT_ID =  # your Foursquare ID
CLIENT_SECRET =  # your Foursquare Secret
VERSION = '20200202'
LIMIT = 10000
radius = 150
# to avoid checking the already visited venues we use dictionar to store list of venues ids 
IDs = defaultdict(list) 
no_nearby_venues=[]
no_of_tips=[]
avg_rating=[]

SyntaxError: invalid syntax (<ipython-input-15-df968602c8c7>, line 2)

In [ ]:
for i in range(len(df)):
#for i in range(0,10):
    print(i)
    #get the nearby venues ids, within 500m
    ids=getNearbyids(df.iloc[i]['latitude'],df.iloc[i]['longitude'],radius)
    ## for each veneue get the number of tips and the rating
    IDs=get_venue_info(ids,IDs)
    ##for each listing get the average rating for the sorrounding venues and the total number of tips
    rating,tips=listing_nearby(ids,IDs)
    no_nearby_venues.append(len(ids))
    avg_rating.append(rating)
    no_of_tips.append(tips)
    

#### adding this information to the datframe

In [ ]:
df['no_nearby_venues']=no_nearby_venues
df['avg_rating']=avg_rating
df['no_of_tips']=no_of_tips

#### combining the old anf the new datframes and store the result

In [ ]:
frames = [dff, df]
result = pd.concat(frames);

In [6]:
#with open("datframe1.txt", "wb") as fp:
#    pickle.dump(result, fp)

with open("datframe1.txt", "rb") as fp:   # Unpickling
    df= pickle.load(fp)

# building the model..

In [7]:
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

In [8]:
df['rent']=df['rent'].astype(float)
df['spaces']=df['spaces'].astype(float)

In [9]:
data_copy=df.dropna()
#dropping irrelevant features
X=data_copy.drop(columns=['price_per_sqft',
                            'AREA_SHORT_CODE',
                          'rent','nearest_subway','location'
                         ])
# Features 
X=X.reset_index(drop=True)
# Targets
y=data_copy['price_per_sqft']


In [10]:
print(len(X.columns))
X.columns

26


Index(['Low_Income_Families', 'Neighbor_area', 'Part_Time_Employment',
       'Pop-Females', 'Pop-Males', 'Pop_10-24', 'Pop_25-39', 'Pop_40-59',
       'Pop_60-89', 'Total_Population', 'avg_crime', 'avg_rating',
       'cafes_within_500m', 'colleges_within_1000m', 'dist_to_bus_km',
       'dist_to_subway_km', 'food_within_500m', 'fun_within_500m',
       'hospitals_within_1000m', 'latitude', 'longitude', 'no_nearby_venues',
       'no_of_tips', 'shopping_within_1500m', 'spaces', 'subway_daily_riders'],
      dtype='object')

#### using MinMaxScaler to scale the features

In [12]:
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,test_size=0.33)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_scaled=scaler.fit_transform(X)

#### Gradient Boosting regressor 

In [13]:
est= GradientBoostingRegressor(random_state=0)
grid_values={'learning_rate': [i for i in np.linspace(0.001, 0.1, num=100)],
              'n_estimators':[int(i) for i in np.linspace(51, 150, num=100)]
             }
clf=GridSearchCV(est, param_grid = grid_values,cv=4).fit(X_scaled,y)

clf.best_score_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.35598244618063146

In [16]:
print(f'Grid best for forest={clf.best_params_} ')

Grid best for forest={'learning_rate': 0.056, 'n_estimators': 52} 


#### to study the feature importances

In [17]:
est= GradientBoostingRegressor(learning_rate= 0.056, n_estimators= 52).fit(X_scaled,y)

In [22]:
[x for _, x in sorted(zip(est.feature_importances_,X.columns), key=lambda pair: -pair[0])]

['cafes_within_500m',
 'dist_to_subway_km',
 'spaces',
 'colleges_within_1000m',
 'longitude',
 'avg_rating',
 'no_nearby_venues',
 'no_of_tips',
 'avg_crime',
 'latitude',
 'Low_Income_Families',
 'dist_to_bus_km',
 'Pop_60-89',
 'Pop_25-39',
 'hospitals_within_1000m',
 'food_within_500m',
 'Pop-Males',
 'shopping_within_1500m',
 'Neighbor_area',
 'fun_within_500m',
 'Pop_10-24',
 'Part_Time_Employment',
 'Pop-Females',
 'Pop_40-59',
 'Total_Population',
 'subway_daily_riders']

###### RidgeCV

In [ ]:
ridge=RidgeCV(alphas=np.logspace(-0.5,0.6,150))
scores = cross_val_score(ridge, X_scaled, y, cv=5)
scores.mean()

### Decision Tree

In [ ]:
est =DecisionTreeRegressor(random_state=0)
grid_values={'min_samples_leaf': [int(i) for i in np.linspace(1, 10, num=10)],
              'max_depth':[int(i) for i in np.linspace(1, 20, num=20)]
             }

clf=GridSearchCV(est, param_grid = grid_values,cv=4).fit(X_scaled,y)

clf.best_score_


### Random Forest

In [ ]:

grid_values={'min_samples_leaf': [int(i) for i in np.linspace(1, 10, num=10)],
              'max_depth':[int(i) for i in np.linspace(1, 20, num=20)]
             }
est=RandomForestRegressor(n_estimators=10,random_state=0)
clf=GridSearchCV(est, param_grid = grid_values,cv=5).fit(X_scaled,y)

clf.best_score_

In [ ]:
print(f'Grid best for forest={clf.best_params_} ')

#### MLP

In [ ]:
from sklearn.neural_network import MLPRegressor
clf = MLPRegressor(hidden_layer_sizes=(10,10,100),random_state=0)
scores = cross_val_score(clf, X_scaled, y, cv=4)
scores.mean()